In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QIcon


# Peice classes
class Piece:
    def __init__(self, colour):
        self.colour = colour
        self.x = int
        self.y = int


class Rook(Piece):
    def __init__(self, colour):
        super().__init__(colour)

    def getValidMoves(self, board, x, y):
        print("rook valid moves")
        valid_moves = []
        valid_moves.append((x, y))

        # Check if the square in front of the rook is empty
        # Also check if a piece is in the way
        for i in range(1, 8):
            if y + i <= 7:
                if board[x][y + i] is None:
                    valid_moves.append((x, y + i))
                elif board[x][y + i].colour != board[x][y].colour:
                    valid_moves.append((x, y + i))
                    break
                else:
                    break
            else:
                break

            if board[x][y + i] is not None:
                break
        # check if the square behind the rook is empty
        for i in range(1, 8):
            if y - i >= 0:
                if board[x][y - i] is None:
                    valid_moves.append((x, y - i))
                elif board[x][y - i].colour != board[x][y].colour:
                    valid_moves.append((x, y - i))
                    break
                else:
                    break
        # check if the square to the left of the rook is empty
        for i in range(1, 8):
            if x + i <= 7:
                if board[x + i][y] is None:
                    valid_moves.append((x + i, y))
                elif board[x + i][y].colour != board[x][y].colour:
                    valid_moves.append((x + i, y))
                    break
                else:
                    break
        # check if the square to the right of the rook is empty
        for i in range(1, 8):
            if x - i >= 0:
                if board[x - i][y] is None:
                    valid_moves.append((x - i, y))
                elif board[x - i][y].colour != board[x][y].colour:
                    valid_moves.append((x - i, y))
                    break
                else:
                    break
        return valid_moves

class Knight(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Bishop(Piece):
    def __init__(self, colour):
        super().__init__(colour)
    
    # TODO fix bug where bishop is duplicated after a move
    def getValidMoves(self, board, x, y):
        valid_moves = []
        # Check diagonal up-right
        i = 1
        while x + i <= 7 and y + i <= 7:
            if board[x + i][y + i] is None:
                valid_moves.append((x + i, y + i))
            elif board[x + i][y + i].colour != self.colour:
                valid_moves.append((x + i, y + i))
                break
            else:
                break
            i += 1
        # Check diagonal up-left
        i = 1
        while x + i <= 7 and y - i >= 0:
            if board[x + i][y - i] is None:
                valid_moves.append((x + i, y - i))
            elif board[x + i][y - i].colour != self.colour:
                valid_moves.append((x + i, y - i))
                break
            else:
                break
            i += 1
        # Check diagonal down-right
        i = 1
        while x - i >= 0 and y + i <= 7:
            if board[x - i][y + i] is None:
                valid_moves.append((x - i, y + i))
            elif board[x - i][y + i].colour != self.colour:
                valid_moves.append((x - i, y + i))
                break
            else:
                break
            i += 1
        # Check diagonal down-left
        i = 1
        while x - i >= 0 and y - i >= 0:
            if board[x - i][y - i] is None:
                valid_moves.append((x - i, y - i))
            elif board[x - i][y - i].colour != self.colour:
                valid_moves.append((x - i, y - i))
                break
            else:
                break
            i += 1
        return valid_moves


class Queen(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class King(Piece):
    def __init__(self, colour):
        super().__init__(colour)


class Pawn(Piece):
    def __init__(self, colour):
        super().__init__(colour)
        self.firstMove = True

    # TODO fix bug where pawn is duplicated when two opposing pawns are able to be captured
    def getValidMoves(self, board, x, y):
        valid_moves = []
        valid_moves.append((x, y))
        if self.colour == "white":
            # Check if the square in front of the pawn is empty
            if board[x + 1][y] is None:
                valid_moves.append((x + 1, y))
                # Check if the pawn is on its first move and if the square two squares in front of the pawn is empty
                if self.firstMove and board[x + 2][y] is None:
                    valid_moves.append((x + 2, y))
            # Check if the square to the left of the pawn is occupied by an enemy piece
            if board[x + 1][y + 1] is not None and board[x + 1][y + 1].colour == "black":
                valid_moves.append((x + 1, y + 1))
            # Check if the square to the right of the pawn is occupied by an enemy piece
            if board[x + 1][y - 1] is not None and board[x + 1][y - 1].colour == "black":
                valid_moves.append((x + 1, y - 1))
        else:
        # Black pawn
    
            # Check if the square in front of the pawn is empty
            if board[x - 1][y] is None:
                valid_moves.append((x - 1, y))
                # Check if the pawn is on its first move and if the square two squares in front of the pawn is empty
                if self.firstMove and board[x - 2][y] is None:
                    valid_moves.append((x - 2, y))
            # Check if the square to the left of the pawn is occupied by an enemy piece
            if board[x - 1][y - 1] is not None and board[x - 1][y - 1].colour == "white":
                valid_moves.append((x - 1, y - 1))
            # Check if the square to the right of the pawn is occupied by an enemy piece
            if board[x - 1][y + 1] is not None and board[x - 1][y + 1].colour == "white":
                valid_moves.append((x - 1, y + 1))
        return valid_moves


class ChessBoard:
    def __init__(self):
        self.moveCount = 0

        self.board = [[None for x in range(8)] for y in range(8)]
        self.buttons = {}
        self.selectedButton = None
        self.highlightedSquares = []

        # Create white pieces
        self.board[0][0] = Rook("white")
        self.board[0][1] = Knight("white")
        self.board[0][2] = Bishop("white")
        self.board[0][3] = Queen("white")
        self.board[0][4] = King("white")
        self.board[0][5] = Bishop("white")
        self.board[0][6] = Knight("white")
        self.board[0][7] = Rook("white") # For development purposes / Debugging
        for i in range(8):
            self.board[1][i] = Pawn("white")

        # Create black pieces
        self.board[7][0] = Rook("black")
        self.board[7][1] = Knight("black")
        self.board[7][2] = Bishop("black")
        self.board[7][3] = Queen("black")
        self.board[7][4] = King("black")
        self.board[7][5] = Bishop("black")
        self.board[7][6] = Knight("black")
        self.board[7][7] = Rook("black")
        for i in range(8):
            self.board[6][i] = Pawn("black")
        
        self.board[3][1] = Bishop("white")
        
        
        #! For development purposes / Debugging
        """
        for x in range(8):
            for y in range(8):
                print(f"{self.board[x][y]} : {x} : {y} : {self.board[x][y].colour if self.board[x][y] is not None else None}")
        """

    # Checks if a piece and a square have been clicked, and calls the movePeice function
    def checkMove(self, piece, square):
        if piece is not None and square is not None:
            x, y = [int(i) for i in square.split(",")]
            if self.selectedButton is None:
                self.selectedButton = self.buttons[square]
                self.selectedButton.setStyleSheet(
                    "background-color: yellow; border: None"
                )
                valid_moves = self.getValidMoves(self.board, x, y)
                self.highlightSquares(piece, valid_moves)
            else:
                # get the coordinates of the previously selected button
                print(f"{self.selectedButton.objectName()} : selected button")


                print(self.buttons.__class__.__name__)
                current_x, current_y = [
                    int(i) for i in self.selectedButton.objectName().split(",")
                ]

                if isinstance(piece, Pawn) and piece.colour == "white" and current_x == 6 and x == 4:
                    # Handle first move of white pawn
                    self.board[4][y] = piece
                    self.board[6][y] = None
                    piece.x = 4
                    piece.y = y
                    self.drawPiece(self.buttons[f"4,{y}"], piece)
                    self.drawPiece(self.buttons[f"6,{y}"], None)
                    self.moveCount += 1
                    self.playerTurn = "Black"
                elif isinstance(piece, Pawn) and piece.colour == "black" and current_x == 1 and x == 3:
                    # Handle first move of black pawn
                    self.board[3][y] = piece
                    self.board[1][y] = None
                    piece.x = 3
                    piece.y = y
                    self.drawPiece(self.buttons[f"3,{y}"], piece)
                    self.drawPiece(self.buttons[f"1,{y}"], None)
                    self.moveCount += 1
                    self.playerTurn = "White"
                else:
                    self.movePiece(piece, current_x, current_y, x, y)
                self.selectedButton.setStyleSheet(
                    "background-color: None; border: None"
                )
                self.selectedButton = None
                self.highlightSquares(
                    piece, [x, y]
                )  # Pass new_x and new_y to highlightSquares
                self.drawSquare(
                    current_x, current_y
                )  # Reset the background color of the previously selected button

    # Draw the board squares
    def drawSquare(self, x, y):
        button = QPushButton()
        button.setFixedSize(100, 100)
        if (x + y) % 2 == 0:
            button.setProperty("class", "white")
            button.setStyleSheet("background-color: white; border: None")
        else:
            button.setProperty("class", "black")
            button.setStyleSheet("background-color: green; border: None")
        
        #! For development purposes / Debugging
        button.setText(f"{x},{y}")

        self.buttons[f"{x},{y}"] = button

        button.setObjectName(
            f"{x},{y}"
        )  # Set the objectName property to the square coordinates

        # Connect the clicked signal of the button to check if a piece and a square have been clicked
        button.clicked.connect(
            lambda _, piece=self.board[x][y], square=button.objectName(): self.checkMove(
                piece, square
            )
        )
        return button

    # Draw the chess board
    def drawBoard(self, layout):
        for x in range(8):
            for y in range(8):
                button = self.drawSquare(x, y)
                layout.addWidget(button, x, y)

        # Check if there is a piece on the square and call drawPiece
        for x in range(8):
            for y in range(8):
                if self.board[x][y] is not None:
                    self.drawPiece(self.buttons[f"{x},{y}"], self.board[x][y])

    # Draw image on a button
    def drawPiece(self, button, piece):
        if piece is not None and piece.__class__.__name__ != "None":
            icon = QIcon(f"media/{piece.colour}/{piece.__class__.__name__}.svg")
            button.setIcon(icon)
            button.setIconSize(button.size())
            button.setObjectName(f"{piece.x},{piece.y}")  # Set the objectName property to the piece coordinates
        else:
            button.setIcon(QIcon())
            
    
    # Calls the getValidMoves function of the selected piece and returns an array of valid moves
    def getValidMoves(self, board, x, y):
        x = int(x)
        y = int(y)
        if self.board[x][y] is not None:
            print(self.board[x][y].getValidMoves(board, x, y))
            return self.board[x][y].getValidMoves(board, x, y)
        else:
            return []

    def highlightSquares(self, piece, squares):
    # get the coordinates of the previously selected button
        current_x = squares[0][0]
        current_y = squares[0][1]
        
        # remove the coordinates of the previously selected button
        squares.pop(0)

        print(f"{self.highlightedSquares} : highlighted squares")
        for button in self.highlightedSquares:
            if button.property("class") == "white":
                button.setStyleSheet("background-color: white; border: None")
            elif button.property("class") == "black":
                button.setStyleSheet("background-color: green; border: None")

            button.clicked.disconnect()
        self.highlightedSquares = []

        # Iterate through the squares array and highlight the squares
        for square in squares:
            if isinstance(square, tuple) and len(square) == 2:
                if square[0] >= 0 and square[0] <= 7 and square[1] >= 0 and square[1] <= 7:
                    button = self.buttons[f"{square[0]},{square[1]}"]
                    self.highlightedSquares.append(button)
                    button.setStyleSheet("background-color: blue; border: None")

                    button.clicked.connect(
                        lambda _, piece=piece, current_x=current_x, current_y=current_y, new_x=square[
                            0
                        ], new_y=square[
                            1
                        ]: self.movePiece(
                            piece, current_x, current_y, new_x, new_y
                        )
                    )

    def movePiece(self, piece, current_x, current_y, new_x, new_y):
        print(f"{self.moveCount} move count") #! For development purposes / Debugging

        # check if the piece is not None and if it is the correct players turn
        if piece is not None:
            # Prevent the player from moving the pawn two squares twice
            if piece.__class__.__name__ == "Pawn":
                piece.firstMove = False

            self.board[current_x][current_y] = None
            self.board[new_x][new_y] = piece

            self.drawPiece(self.buttons[f"{new_x},{new_y}"], piece)
            self.drawPiece(self.buttons[f"{current_x},{current_y}"], None)
            
            if self.buttons[f"{current_x},{current_y}"].icon() is not None:
                self.drawPiece(self.buttons[f"{current_x},{current_y}"], None)
                # increase move count
                self.moveCount += 1
                self.playerTurn = "White" if self.moveCount % 2 == 0 else "Black"
                self.highlightSquares(
                    piece, self.getValidMoves(self.board, new_x, new_y)
                )
            else:
                print("invalid move")
        else:
            print("invalid move")
            self.highlightSquares(None, [])
        
        for x in range(8):
            for y in range(8):
                print(self.board[x][y])
        # check for pawn promotion
        #self.checkForPromotion()

# TODO finish pawn promotion window / UI
"""
    def checkForPromotion(self):
        # white pawn promotion
        for x in range(8):
            if isinstance(self.board[7][x], Pawn):
                self.board[7][x] = getPromotionPiece("white", x=x, y=7)
                self.drawPiece(self.buttons[f"7,{x}"], self.board[7][x])
    


# Promotion UI
class PromotionWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        grid = QGridLayout()
        self.setLayout(grid)

        # Create buttons for each piece
        queen_btn = QPushButton('Queen')
        queen_btn.setIcon(QIcon('media/white/Queen.svg'))
        queen_btn.clicked.connect(lambda: self.piece_selected('queen'))

        rook_btn = QPushButton('Rook')
        rook_btn.setIcon(QIcon('media/white/Rook.svg'))
        rook_btn.clicked.connect(lambda: self.piece_selected('rook'))

        bishop_btn = QPushButton('Bishop')
        bishop_btn.setIcon(QIcon('media/white/Bishop.svg'))
        bishop_btn.clicked.connect(lambda: self.piece_selected('bishop'))

        knight_btn = QPushButton('Knight')
        knight_btn.setIcon(QIcon('media/white/Knight.svg'))
        knight_btn.clicked.connect(lambda: self.piece_selected('knight'))

        # Add buttons to grid layout
        grid.addWidget(queen_btn, 0, 0)
        grid.addWidget(rook_btn, 0, 1)
        grid.addWidget(bishop_btn, 1, 0)
        grid.addWidget(knight_btn, 1, 1)

        self.setGeometry(100, 100, 200, 100)
        self.setWindowTitle('Promotion')
        self.show()

    def piece_selected(self, piece):
        self.close()
        return piece

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PromotionWindow()
    sys.exit(app.exec_())
"""

# UI
class MainWindow(QWidget):
    def __init__(self):
        QWidget.__init__(self)
        self.windowTitle = "Chess"

        # Create layout
        layout = QGridLayout()
        self.setLayout(layout)

        # Create chess board

        board = ChessBoard()
        board.drawBoard(layout)

if __name__ == "__main__":
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec()

[(4, 2), (5, 3), (6, 4), (4, 0), (2, 2), (2, 0)]
[] : highlighted squares
<class 'int'>,<class 'int'> : selected button
dict


ValueError: invalid literal for int() with base 10: "<class 'int'>"

0 move count
[(7, 5), (7, 3), (5, 5), (4, 6), (3, 7), (5, 3), (4, 2)]
[<PyQt5.QtWidgets.QPushButton object at 0x7faef80243a0>, <PyQt5.QtWidgets.QPushButton object at 0x7faef80248b0>, <PyQt5.QtWidgets.QPushButton object at 0x7faf082e7d00>, <PyQt5.QtWidgets.QPushButton object at 0x7faf082e7520>, <PyQt5.QtWidgets.QPushButton object at 0x7faf082e7400>] : highlighted squares
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<class 'int'>,<class 'int'> : selected button
dict


ValueError: invalid literal for int() with base 10: "<class 'int'>"

1 move count
[(6, 2)]
[<PyQt5.QtWidgets.QPushButton object at 0x7faef8024ca0>, <PyQt5.QtWidgets.QPushButton object at 0x7faef80244c0>, <PyQt5.QtWidgets.QPushButton object at 0x7faef80240d0>, <PyQt5.QtWidgets.QPushButton object at 0x7faf082e7c70>, <PyQt5.QtWidgets.QPushButton object at 0x7faef80243a0>, <PyQt5.QtWidgets.QPushButton object at 0x7faf082e7e20>] : highlighted squares
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
